In [1]:
import warnings
warnings.filterwarnings('ignore')
    
import pandas as pd
import numpy as np
import datetime
price_raw = pd.read_csv('dataset/minuteByminutePrice/priceDataJanuary2019.csv')
news_raw = pd.read_csv('dataset/minuteByminutePrice/data_jan_2019_analysis.csv')


In [2]:
# Convert news to minute by minute dataset
news_data = news_raw[['SENTIMENT_VALUE','relevance','pub_date']]
news_data['pub_date'] =  pd.to_datetime(news_data['pub_date'], format='%Y-%m-%d %H:%M:%S')

def round_timestamp(t):
    t = datetime.datetime(t.year, t.month, t.day, t.hour, t.minute, 0)  
    return t

news_data['timestamp_rounded'] = news_data['pub_date'].apply(round_timestamp)
news_data.head()

,SENTIMENT_VALUE,relevance,pub_date,timestamp_rounded
0,-0.4,1,2019-01-01 00:36:31,2019-01-01 00:36:00
1,-1.0,1,2019-01-01 00:45:40,2019-01-01 00:45:00
2,-0.9,1,2019-01-01 01:04:01,2019-01-01 01:04:00
3,1.0,1,2019-01-01 01:44:15,2019-01-01 01:44:00
4,0.9,1,2019-01-01 05:18:48,2019-01-01 05:18:00


In [3]:
# Aggregate news_data
news_data['value'] = news_data['SENTIMENT_VALUE'] *  news_data['relevance'] 
# news_data.groupby('pub_date_rounded').agg({'SENTIMENT_VALUE':['sum', 'max','count'], 
#                          'relevance': lambda x: x.max() - x.min()})
# 2019-01-01 10:00:00	
news_data_aggregated = news_data.groupby('timestamp_rounded').agg({'value':['mean', 'count']})
news_data_aggregated.head(10)

value      
                         mean count
timestamp_rounded                  
2019-01-01 00:36:00 -0.400000     1
2019-01-01 00:45:00 -1.000000     1
2019-01-01 01:04:00 -0.900000     1
2019-01-01 01:44:00  1.000000     1
2019-01-01 05:18:00  0.900000     1
2019-01-01 08:24:00 -0.700000     1
2019-01-01 10:00:00  0.333333     3
2019-01-01 13:37:00 -1.000000     1
2019-01-01 15:28:00  1.000000     1
2019-01-01 15:43:00 -0.900000     1

In [4]:
# process price_data
price_raw['timestamp'] = price_raw['Date'] + ' ' + price_raw['Time']
price_raw['timestamp'] =  pd.to_datetime(price_raw['timestamp'], format='%Y-%m-%d %H:%M:%S')
price_raw['timestamp_rounded'] = price_raw['timestamp'].apply(round_timestamp)

price_raw.head()

,Unnamed: 0,Open,High,Low,Close,Volume,Time,Date,timestamp,timestamp_rounded
0,3676576,3693.85,3698.79,3693.85,3698.00,5.491289,00:00:00,2019-01-01,2019-01-01 00:00:00,2019-01-01 00:00:00
1,3676577,3694.72,3694.72,3690.65,3690.65,9.500151,00:01:00,2019-01-01,2019-01-01 00:01:00,2019-01-01 00:01:00
2,3676578,3689.73,3689.73,3686.62,3686.62,0.965966,00:02:00,2019-01-01,2019-01-01 00:02:00,2019-01-01 00:02:00
3,3676579,3692.85,3692.85,3688.32,3692.35,0.296662,00:03:00,2019-01-01,2019-01-01 00:03:00,2019-01-01 00:03:00
4,3676580,3692.35,3692.35,3690.34,3690.34,0.111622,00:04:00,2019-01-01,2019-01-01 00:04:00,2019-01-01 00:04:00


In [5]:
price_data_aggregated = price_raw.groupby('timestamp_rounded').agg({
    'Open':['mean'],
    'High':['max'],
    'Low':['min'],
    'Close':['mean'],
    'Volume':['mean'] 
})

price_raw.count()

Unnamed: 0           40142
Open                 40142
High                 40142
Low                  40142
Close                40142
Volume               40142
Time                 40142
Date                 40142
timestamp            40142
timestamp_rounded    40142
dtype: int64

In [6]:
final_data = price_data_aggregated.join(news_data_aggregated, how='outer', on = ['timestamp_rounded'])
final_data.count()
final_data = final_data.reset_index()
final_data.columns = ['Timestamp_Rounded', 'Open','High','Low','Close','Volume','Sentiment_Value','news_count']
# Fill null values for news with 0 (neutrial news ?!)
final_data['news_count'] = final_data['news_count'].fillna(0)
final_data['Sentiment_Value'] = final_data['Sentiment_Value'].fillna(0)
final_data

,Timestamp_Rounded,Open,High,Low,Close,Volume,Sentiment_Value,news_count
0,2019-01-01 00:00:00,3693.85,3698.79,3693.85,3698.00,5.491289,0.0,0.0
1,2019-01-01 00:01:00,3694.72,3694.72,3690.65,3690.65,9.500151,0.0,0.0
2,2019-01-01 00:02:00,3689.73,3689.73,3686.62,3686.62,0.965966,0.0,0.0
3,2019-01-01 00:03:00,3692.85,3692.85,3688.32,3692.35,0.296662,0.0,0.0
4,2019-01-01 00:04:00,3692.35,3692.35,3690.34,3690.34,0.111622,0.0,0.0
5,2019-01-01 00:05:00,3690.40,3690.85,3690.40,3690.85,2.247676,0.0,0.0
6,2019-01-01 00:06:00,3688.61,3688.61,3687.08,3687.08,1.556315,0.0,0.0
7,2019-01-01 00:07:00,3684.59,3685.52,3684.59,3684.77,0.823009,0.0,0.0
8,2019-01-01 00:08:00,3684.97,3688.79,3683.45,3683.45,4.493121,0.0,0.0
9,2019-01-01 00:09:00,3685.00,3685.12,3684.58,3685.12,1.077187,0.0,0.0


In [7]:
final_data.to_csv('dataset/aggregated/min_by_min/jan.csv')